In [5]:
import pandas as pd
bad = pd.read_csv('../data/bad/bad.csv')
normal=pd.read_csv('../data/neautral/neutral.csv')
hiniku_=pd.read_csv('../data/hiniku/hiniku_with_word.csv')

In [6]:
positivelist = list(hiniku_.dropna().positiveword.values)
hinikulist = list(hiniku_.dropna().hinikuword.values)
badwordlist = list(bad.word)#悪口単語をリストで抽出

In [107]:
'''
悪口ワードの含有
皮肉ワードの含有
ポジティブワードの含有

[形態素解析した時の単語数],[単純に悪口単語が文章に含まれているのかのフラグ], [形態素ごとに現れる悪口数]
[皮肉単語が含まれているかのフラグ],[形態素ごとに現れる皮肉数]
[ポジティブな単語が含まれているかのフラグ], [形態素ごとに現れるポジティブ単語数]

'''
import pandas as pd
# df = bad.copy()
df = pd.concat([hiniku_, normal])
df = pd.concat([bad, df])
import sys
sys.path.append('/Users/nagataeiki/.pyenv/versions/3.7.4/lib/python3.7/site-packages')
import numpy as np
from janome.tokenizer import Tokenizer
from tqdm import tqdm

tokenizer = Tokenizer()
#悪口が含まれている数のカウント, 単に含まれているのかのフラグを返す
def get_word_count(text, wordlist):
    textlist = []
    count=0
    flg=0
    for t in tokenizer.tokenize(text):
        if t.surface in wordlist:
            count+=1
    for w in wordlist:
        if w in text:
            flg=1
            return count, flg
    
    return count,flg


#dfと悪口辞書を入れて新しい特徴量を含んだdfを返す
def make_bad_feature(df, badwordlist, positivelist, hinikulist):
    col=["content","mophologics_num", "bad_per_mophologic", "simple_bad_flg","target", "hiniku_per_mophologic",\
         "simple_hiniku_flg", "positive_per_mophologic", "simple_positive_flg"]
    basedf = pd.DataFrame(columns=col)
    tokenizer = Tokenizer()
    for r in tqdm(df.iterrows()):
        text = str(r[1].content)
        target = r[1].type
        num = len(tokenizer.tokenize(text))
        badcount,badflg = get_word_count(text, badwordlist)
        hinikucount, hinikuflg = get_word_count(text, hinikulist)
        positivecount, positiveflg = get_word_count(text, positivelist)
        newdf = pd.DataFrame(np.array([text, num, badcount,badflg,target,\
                                       hinikucount, hinikuflg, positivecount,\
                                       positiveflg]).reshape(1,-1),columns=col)
        basedf = pd.concat([basedf, newdf])

    basedf["target"] = basedf.apply(lambda x: int(x["target"]), axis=1)
    return basedf
df =make_bad_feature(df, badwordlist, positivelist, hinikulist)
df.head(2)
print(df.shape)


750it [00:06, 109.38it/s]

(750, 9)


In [14]:
# df.head()

In [12]:
# hiniku_ = pd.read_csv('../data/hiniku/hiniku_with_word.csv')

# col=["content","mophologics_num", "bad_word_num", "simple_bad_flg","target", "hinikuword", "positiveword"]


In [22]:
# '''
# 1対1で予測をする時
# '''

# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.metrics import f1_score
# x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
# svm = SVC(C=1.0, random_state=42)
# svm.fit(x_train, y_train)

# pred_train = svm.predict(x_train)
# pred_test = svm.predict(x_test)
# print(f1_score(y_train, pred_train))
# print(f1_score(y_test, pred_test))

In [142]:
"""
1対その他での学習
"""


from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import f1_score


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def vectorize(x_train,x_test, vec):
    x_train_vec = vec.fit_transform(x_train.content)
    x_test_vec = vec.transform(x_test.content)
    
    x_train_vec = pd.DataFrame(x_train_vec.todense())
    x_test_vec = pd.DataFrame(x_test_vec.todense())
    
    x_train_vec["comment_id"]=x_train.comment_id.values
    x_test_vec["comment_id"]=x_test.comment_id.values
    
    return x_train_vec,x_test_vec



def train_and_predict(df):

    X = df.drop("target",axis=1)
    y = df.target

    x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
    
    vec = TfidfVectorizer()
    x_train_vec, x_test_vec = vectorize(x_train, x_test, vec)
    x_train.drop("content",axis=1,inplace=True)
    x_test.drop("content",axis=1,inplace=True)
    new_x_train = x_train.merge(x_train_vec, on="comment_id")#.drop('content',axis=1)
    new_x_test  = x_test.merge(x_test_vec, on="comment_id")#.drop('content',axis=1)
    svm = SVC(C=1.0, random_state=42)
    model = OneVsRestClassifier(svm)
    model.fit(new_x_train, y_train)
    print('fin')

    return {
        "pred_train":  model.predict(new_x_train),
        "pred_test": model.predict(new_x_test),
        "x_train":new_x_train,
        "y_train":y_train,
        "x_test":new_x_test,
        "y_test":y_test,
#         "x_train_vec":x_train_vec,
#         "x_test_vec":x_test_vec
    }

In [127]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import numpy as np
'''
mxm,ここでは3x3のマトリックスを作成する
そこからf1値を計算する（同ディレクトリの画像参照）
'''

def create_3x3_matrix(true_y, pred_y):
    matrix = np.zeros(9)

    for t,p in zip(true_y, pred_y):
        if t==0:
            if p==0:matrix[0]+=1
            if p==1:matrix[1]+=1
            if p==2:matrix[2]+=1
        elif t==1:
            if p==0:matrix[3]+=1
            if p==1:matrix[4]+=1
            if p==2:matrix[5]+=1
        elif t==2:
            if p==0:matrix[6]+=1
            if p==1:matrix[7]+=1
            if p==2:matrix[8]+=1

    return matrix.reshape(3,3)
# matrix=create_3x3_matrix(y_train, pred_train)
# matrix

In [130]:
'''
 calc_scores(matrix)
 
{'recalls': array([0.31693989, 0.34710744, 0.28      ]),
 'Precisions': array([0.28292683, 0.41176471, 0.2565445 ]),
 'f1': 0.31587601668402826}
'''

def calc_scores(matrix):
    Recalls = np.zeros(3)
    for i in range(3):
        Recalls[i] = matrix[i][i]/sum(matrix[i])
    Precisions = np.zeros(3)
    pred_sum = matrix.sum(axis=0)
    for i, ps in enumerate(pred_sum):
        Precisions[i] = matrix[i][i]/ps
    f1_score = 2*Recalls.mean()*Precisions.mean()/(Recalls.mean()+Precisions.mean())
    return {
        "recalls":Recalls,
        "Precisions":Precisions,
        "f1":f1_score
    }

In [131]:
'''
predのキー
dict_keys(['pred_train', 'pred_test', 'x_train', 'y_train', 'x_test', 'y_test'])
'''

def print_miss_pred(pred, test=True):
    if test:
        true_y = pred["y_test"]
        pred_y = pred["pred_test"]
        df     = pred["x_test"]
    else:
        true_y = pred["y_train"]
        pred_y = pred["pred_train"]
        df     = pred["x_train"]
        
    idx_arr = []
    pred_judge=[]
    true_judge=[]
    for i in range(len(pred)):
        if pred[i]!=true_y.values[i]:
#             print(pred[i], true_y.values[i])
            idx_arr.append(i)
            pred_judge.append(pred[i])
            true_judge.append(true_y.values[i])

    def print_str_label(num):
        if num==0:return "ニュートラル"
        if num==1:return "悪口"
        if num==2:return "皮肉"
        return np.nan

    for i,idx in enumerate(idx_arr):
        display(df.iloc[idx].content)
        print('true label  *',true_judge[i] , (print_str_label(true_judge[i])))
        print('pred label  *',pred_judge[i], (print_str_label(pred_judge[i])))
        print("------------------------------------------------------------------------------------")
#     return idx_arr
#     return pred_judge, true_judge
# print_miss_pred(y_test, pred_test)
# p,t = print_miss_pred(y_test, pred_test)

In [133]:
# df1.to_csv('../data/clean/mount_data.csv', index=False)
# df1.to_excel('../data/clean/mount_data.xlsx', index=False)
# df1 = pd.read_excel('../data/clean/mount_data.xlsx')
# df1 = df1.dropna()
# df1["type"]=df1.target
# df1 = make_bad_feature(df1, badwordlist=badwordlist, positivelist=positivelist, hinikulist=hinikulist)

In [134]:
# df1[df1.target==0].shape[0], df1[df1.target==1].shape[0],df1[df1.target==2].shape[0]

In [135]:
# train_matrix = create_3x3_matrix(pred["y_train"], pred["pred_train"])
# test_matrix  = create_3x3_matrix(pred['y_test'],  pred['pred_test'])

In [136]:
# calc_scores(train_matrix)

In [137]:
# calc_scores(test_matrix)

In [213]:
# print_miss_pred(pred)

In [139]:
# '''
# predのキー
# dict_keys(['pred_train', 'pred_test', 'x_train', 'y_train', 'x_test', 'y_test'])
# '''

# def print_miss_pred(pred, test=True, true_y=np.nan, pred_y=np.nan,df=np.nan):
#     if test:
#         true_y = pred["y_test"]
#         pred_y = pred["pred_test"]
#         df     = pred["x_test"]
#     else:
#         true_y = pred["y_train"]
#         pred_y = pred["pred_train"]
#         df     = pred["x_train"]


#     idx_arr = []
#     pred_judge=[]
#     true_judge=[]
#     for i in range(len(pred_y)):
#         if pred_y[i]!=true_y.values[i]:
# #             print(pred[i], true_y.values[i])
#             idx_arr.append(i)
#             pred_judge.append(pred_y[i])
#             true_judge.append(true_y.values[i])

#     def print_str_label(num):
#         if num==0:return "ニュートラル"
#         if num==1:return "悪口"
#         if num==2:return "皮肉"
#         return np.nan

#     for i,idx in enumerate(idx_arr):
#         display(df.iloc[idx].content)
#         print('true label  *',true_judge[i] , (print_str_label(true_judge[i])))
#         print('pred label  *',pred_judge[i], (print_str_label(pred_judge[i])))
#         print("------------------------------------------------------------------------------------")
# #     return idx_arr
# #     return pred_judge, true_judge

In [144]:
import pandas as pd
df = pd.read_csv('../data/clean/mount_data.csv').dropna()
df["comment_id"] = df.index

In [140]:
# def tokenize(text):
#     return tokenizer.tokenize(text,wakati=True)
# text="これは美味しいです"
# tokenize(text)

In [203]:
# content = df1.content.values
# target = df1.target.values

In [141]:
'''

'''
# x_train,x_test, y_train, y_test = train_test_split(content, target, random_state=42)
# print('binary')
# vec = CountVectorizer(binary=True)
# train_and_val(x_train, x_test, y_train, y_test, vec)

# print('count')
# vec = CountVectorizer(binary=False)
# train_and_val(x_train, x_test, y_train, y_test, vec)

# print('tfidf')
# vec = TfidfVectorizer()
# vectorize(x_train, x_test, y_train, y_test, vec)

# print("bigram")
# vec = TfidfVectorizer(ngram_range=(1,3))
# train_and_val(x_train, x_test, y_train, y_test, vec)


'\nベクトル化\n'

In [145]:
df

,content,mophologics_num,bad_per_mophologic,simple_bad_flg,target,hiniku_per_mophologic,simple_hiniku_flg,positive_per_mophologic,simple_positive_flg,comment_id
0,ガイジ,1,1,1,1,0,0,0,0,0
1,くどい,1,1,1,1,0,0,0,0,1
2,偽物 ただの釣り動画,6,2,1,1,0,0,0,0,2
3,振り付けダサイ,2,0,1,1,0,0,0,0,3
4,男きも,3,0,1,1,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...
2347,動画もいいな,4,0,0,0,0,0,1,1,2347
2348,3桁記念かきこ,5,0,0,0,0,0,0,0,2348
2349,鳥肌やばい,2,0,0,0,0,0,0,0,2349
2350,兄弟愛に泣いた,5,0,0,0,0,0,0,0,2350


In [147]:
pred = train_and_predict(df)

/Users/nagataeiki/opt/anaconda3/envs/learning/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


fin


In [150]:
matrix=create_3x3_matrix(pred["y_train"], pred["pred_train"])
print(matrix)
calc_scores(matrix)

[[592.   0.   0.]
 [  0. 741.   5.]
 [ 41.   0. 333.]]


{'recalls': array([1.        , 0.99329759, 0.89037433]),
 'Precisions': array([0.93522907, 1.        , 0.9852071 ]),
 'f1': 0.9673125359877756}

In [151]:
matrix=create_3x3_matrix(pred["y_test"], pred["pred_test"])
print(matrix)
calc_scores(matrix)

[[155.   0.   0.]
 [  0. 184.   2.]
 [  4.   0.  83.]]


{'recalls': array([1.        , 0.98924731, 0.95402299]),
 'Precisions': array([0.97484277, 1.        , 0.97647059]),
 'f1': 0.98242878020966}